In [1]:
import os
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm

# Train with Aug + Balance

In [15]:
data_dir = 'datasetWith'
num_classes = 9
batch_size = 32
num_epochs = 20
learning_rate = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_type = "efficientnet"  # Change to "efficientnet"

In [16]:
image_size = 224
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}


In [17]:
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'val']
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True)
    for x in ['train', 'val']
}

class_names = image_datasets['train'].classes
print("Classes:", class_names)


Classes: ['Automatic Rifle', 'Bazooka', 'Grenade Launcher', 'Handgun', 'Knife', 'SMG', 'Shotgun', 'Sniper', 'Sword']


In [18]:
def build_model(model_type="resnet", num_classes=9):
    if model_type == "resnet":
        model = models.resnet50(pretrained=True)
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, num_classes)
    elif model_type == "efficientnet":
        from torchvision.models import efficientnet_b0
        model = efficientnet_b0(pretrained=True)
        in_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(in_features, num_classes)
    else:
        raise ValueError("Invalid model type.")
    return model

model = build_model(model_type=model_type, num_classes=num_classes).to(device)


c:\Users\ma.aldawsari\AppData\Local\anaconda3\envs\p\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\ma.aldawsari/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:01<00:00, 20.3MB/s]


In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [20]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=10):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print('-' * 30)

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloaders[phase]):
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            scheduler.step()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Save best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    print(f"\nBest Validation Acc: {best_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model

model = train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=num_epochs)


Epoch 1/20
------------------------------


100%|██████████| 31/31 [00:10<00:00,  2.98it/s]


Train Loss: 2.0538 Acc: 0.2965


100%|██████████| 4/4 [00:01<00:00,  2.64it/s]


Val Loss: 1.7918 Acc: 0.5243

Epoch 2/20
------------------------------


...

Val Loss: 0.9428 Acc: 0.7379

Epoch 20/20
------------------------------


100%|██████████| 31/31 [00:09<00:00,  3.14it/s]


Train Loss: 0.8064 Acc: 0.8620


100%|██████████| 4/4 [00:01<00:00,  2.69it/s]

Val Loss: 0.9416 Acc: 0.7282

Best Validation Acc: 0.7476


In [21]:
torch.save(model.state_dict(), f"best_weapon_model_{model_type}.pth")
print("Model saved.")

Model saved.


In [10]:
# Load model again later
model = build_model(model_type="resnet", num_classes=9)
model.load_state_dict(torch.load("best_weapon_model_resnet.pth"))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), data_transforms['val'])
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

correct = 0
total = 0
# Move model to device
model.to(device)
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"✅ Test Accuracy: {100 * correct / total:.2f}%")



✅ Test Accuracy: 88.35%


# Predict on Unlabeled Test Images

In [28]:
import os
from PIL import Image, ImageSequence
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# ============ CONFIG ============ #
folder_path = 'datasetWith/test2'
model_path = 'best_weapon_model_resnet.pth'  # or EfficientNet path
model_type = 'resnet'  # or 'efficientnet'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size = 224
batch_size = 1

# Class names (adjust if needed)
class_names = [
    'Automatic Rifle', 'Bazooka', 'Grenade Launcher', 'Handgun', 'Knife',
    'Shotgun', 'SMG', 'Sniper', 'Sword'
]

# ============ TRANSFORM ============ #
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ============ DATASET CLASS ============ #
class UnlabeledImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.image_paths = sorted([
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))
        ])
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def load_gif_first_frame(self, path):
        with Image.open(path) as im:
            frame = next(ImageSequence.Iterator(im)).convert('RGB')
        return frame

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        if path.lower().endswith('.gif'):
            image = self.load_gif_first_frame(path)
        else:
            image = Image.open(path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, os.path.basename(path)

# ============ MODEL LOADING ============ #
from torchvision import models

def build_model(model_type="resnet", num_classes=9):
    if model_type == "resnet":
        model = models.resnet50(pretrained=False)
        model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    elif model_type == "efficientnet":
        from torchvision.models import efficientnet_b0
        model = efficientnet_b0(pretrained=False)
        model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)
    else:
        raise ValueError("Unsupported model type.")
    return model

model = build_model(model_type=model_type, num_classes=len(class_names))
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# ============ PREDICTION ============ #
dataset = UnlabeledImageDataset(folder_path, transform=transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

print("🔎 Predictions:")
with torch.no_grad():
    for inputs, filenames in loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1)
        predicted_class = class_names[preds.item()]
        confidence = probs[0][preds.item()].item() * 100
        print(f"{filenames[0]} → {predicted_class} ({confidence:.2f}%)")


🔎 Predictions:
army.jpg → Sniper (35.34%)
giphy.gif → Sniper (62.39%)
mafia-mafia-game.gif → Bazooka (39.28%)
weapons.jpg → Shotgun (77.84%)
weapons2.jpg → Sniper (72.11%)
weaponsgta5.gif → Automatic Rifle (67.54%)


🔎 Predictions:
army.jpg → Sniper (31.15%)
giphy.gif → Sniper (28.78%)
mafia-mafia-game.gif → Knife (26.18%)
weapons.jpg → Shotgun (24.52%)
weapons2.jpg → Sniper (44.57%)
weaponsgta5.gif → Shotgun (25.95%)

-----

In [41]:
import os
from PIL import Image, ImageSequence
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# ============ CONFIG ============ #
folder_path = 'test2'
model_path = 'best_weapon_model_efficientnet.pth'  # or EfficientNet path
model_type = 'efficientnet'  # or 'efficientnet'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size = 224
batch_size = 1

# Class names (adjust if needed)
class_names = [
    'Automatic Rifle', 'Bazooka', 'Grenade Launcher', 'Handgun', 'Knife',
    'Shotgun', 'SMG', 'Sniper', 'Sword'
]

# ============ TRANSFORM ============ #
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ============ DATASET CLASS ============ #
class UnlabeledImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.image_paths = sorted([
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))
        ])
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def load_gif_first_frame(self, path):
        with Image.open(path) as im:
            frame = next(ImageSequence.Iterator(im)).convert('RGB')
        return frame

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        if path.lower().endswith('.gif'):
            image = self.load_gif_first_frame(path)
        else:
            image = Image.open(path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, os.path.basename(path)

# ============ MODEL LOADING ============ #
from torchvision import models

def build_model(model_type="resnet", num_classes=9):
    if model_type == "resnet":
        model = models.resnet50(pretrained=False)
        model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    elif model_type == "efficientnet":
        from torchvision.models import efficientnet_b0
        model = efficientnet_b0(pretrained=False)
        model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)
    else:
        raise ValueError("Unsupported model type.")
    return model

model = build_model(model_type=model_type, num_classes=len(class_names))
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# ============ PREDICTION ============ #
dataset = UnlabeledImageDataset(folder_path, transform=transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

print("🔎 Predictions:")
with torch.no_grad():
    for inputs, filenames in loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1)
        predicted_class = class_names[preds.item()]
        confidence = probs[0][preds.item()].item() * 100
        print(f"{filenames[0]} → {predicted_class} ({confidence:.2f}%)")


🔎 Predictions:
3d Model Bazooka Rocket Launcher.png → SMG (27.55%)
Handgun _ Self-defense, Concealed Carry, Home Protection _ Britannica.jpg → Handgun (91.16%)
Image.jpg → Handgun (45.00%)


# Without Aug just Balance

In [31]:
data_dir = 'datasetWithB'
num_classes = 9
batch_size = 32
num_epochs = 20
learning_rate = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_type = "resnet"  # Change to "efficientnet"

In [32]:
image_size = 224
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}


In [33]:
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'val']
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True)
    for x in ['train', 'val']
}

class_names = image_datasets['train'].classes
print("Classes:", class_names)


Classes: ['Automatic Rifle', 'Bazooka', 'Grenade Launcher', 'Handgun', 'Knife', 'SMG', 'Shotgun', 'Sniper', 'Sword']


In [34]:
def build_model(model_type="resnet", num_classes=9):
    if model_type == "resnet":
        model = models.resnet50(pretrained=True)
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, num_classes)
    elif model_type == "efficientnet":
        from torchvision.models import efficientnet_b0
        model = efficientnet_b0(pretrained=True)
        in_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(in_features, num_classes)
    else:
        raise ValueError("Invalid model type.")
    return model

model = build_model(model_type=model_type, num_classes=num_classes).to(device)


In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [36]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=10):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print('-' * 30)

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloaders[phase]):
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            scheduler.step()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Save best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    print(f"\nBest Validation Acc: {best_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model

model = train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=num_epochs)


Epoch 1/20
------------------------------


100%|██████████| 11/11 [00:07<00:00,  1.54it/s]


Train Loss: 1.8072 Acc: 0.4225


100%|██████████| 4/4 [00:01<00:00,  2.31it/s]


Val Loss: 1.6647 Acc: 0.4369

Epoch 2/20
------------------------------


...

Val Loss: 1.1072 Acc: 0.6699

Epoch 20/20
------------------------------


100%|██████████| 11/11 [00:06<00:00,  1.80it/s]


Train Loss: 0.1562 Acc: 0.9939


100%|██████████| 4/4 [00:01<00:00,  2.63it/s]

Val Loss: 1.0996 Acc: 0.6602

Best Validation Acc: 0.6990


In [37]:
torch.save(model.state_dict(), f"best_weapon_model_{model_type}.pth")
print("Model saved.")

Model saved.


In [43]:
import os
from PIL import Image, ImageSequence
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# ============ CONFIG ============ #
folder_path = 'test2'
model_path = 'best_weapon_model_resnet.pth'  # or EfficientNet path
model_type = 'resnet'  # or 'efficientnet'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size = 224
batch_size = 1

# Class names (adjust if needed)
class_names = [
    'Automatic Rifle', 'Bazooka', 'Grenade Launcher', 'Handgun', 'Knife',
    'Shotgun', 'SMG', 'Sniper', 'Sword'
]

# ============ TRANSFORM ============ #
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ============ DATASET CLASS ============ #
class UnlabeledImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.image_paths = sorted([
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))
        ])
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def load_gif_first_frame(self, path):
        with Image.open(path) as im:
            frame = next(ImageSequence.Iterator(im)).convert('RGB')
        return frame

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        if path.lower().endswith('.gif'):
            image = self.load_gif_first_frame(path)
        else:
            image = Image.open(path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, os.path.basename(path)

# ============ MODEL LOADING ============ #
from torchvision import models

def build_model(model_type="resnet", num_classes=9):
    if model_type == "resnet":
        model = models.resnet50(pretrained=False)
        model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    elif model_type == "efficientnet":
        from torchvision.models import efficientnet_b0
        model = efficientnet_b0(pretrained=False)
        model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)
    else:
        raise ValueError("Unsupported model type.")
    return model

model = build_model(model_type=model_type, num_classes=len(class_names))
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# ============ PREDICTION ============ #
dataset = UnlabeledImageDataset(folder_path, transform=transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

print("🔎 Predictions:")
with torch.no_grad():
    for inputs, filenames in loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1)
        predicted_class = class_names[preds.item()]
        confidence = probs[0][preds.item()].item() * 100
        print(f"{filenames[0]} → {predicted_class} ({confidence:.2f}%)")


🔎 Predictions:
3d Model Bazooka Rocket Launcher.png → Bazooka (33.78%)
Handgun _ Self-defense, Concealed Carry, Home Protection _ Britannica.jpg → Handgun (46.24%)
Image.jpg → SMG (57.03%)


# Freeze

In [57]:
data_dir = 'datasetWithA&B'
num_classes = 9
batch_size = 32
num_epochs = 20
learning_rate = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_type = "resnet"  # Change to "efficientnet"

In [58]:
image_size = 224
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
}

In [59]:
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'val']
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True)
    for x in ['train', 'val']
}

class_names = image_datasets['train'].classes
print("Classes:", class_names)

Classes: ['Automatic Rifle', 'Bazooka', 'Grenade Launcher', 'Handgun', 'Knife', 'SMG', 'Shotgun', 'Sniper', 'Sword']


In [60]:
def build_model(model_type="resnet", num_classes=9, freeze_backbone=True):
    if model_type == "resnet":
        model = models.resnet50(pretrained=True)
        if freeze_backbone:
            for param in model.parameters():
                param.requires_grad = False
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, num_classes)
        if freeze_backbone:
            for param in model.fc.parameters():
                param.requires_grad = True

    elif model_type == "efficientnet":
        from torchvision.models import efficientnet_b0
        model = efficientnet_b0(pretrained=True)
        if freeze_backbone:
            for param in model.features.parameters():
                param.requires_grad = False
        in_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(in_features, num_classes)
        if freeze_backbone:
            for param in model.classifier[1].parameters():
                param.requires_grad = True

    else:
        raise ValueError("Invalid model type.")
    return model


model = build_model(model_type=model_type, num_classes=num_classes).to(device)

In [61]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [63]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=10):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print('-' * 30)

        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloaders[phase]):
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            scheduler.step()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Save best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
                torch.save(best_model_wts, 'best_model_resnet_version_2.pth')  # Save here

    print(f"\nBest Validation Acc: {best_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model


model = train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=num_epochs)


Epoch 1/20
------------------------------


100%|██████████| 31/31 [00:09<00:00,  3.15it/s]


Train Loss: 2.1589 Acc: 0.1789


100%|██████████| 4/4 [00:01<00:00,  2.38it/s]


Val Loss: 2.0739 Acc: 0.3495

Epoch 2/20
------------------------------


...

Val Loss: 1.7513 Acc: 0.5922

Epoch 20/20
------------------------------


100%|██████████| 31/31 [00:09<00:00,  3.29it/s]


Train Loss: 1.7468 Acc: 0.6268


100%|██████████| 4/4 [00:01<00:00,  2.38it/s]

Val Loss: 1.7538 Acc: 0.5825

Best Validation Acc: 0.6408


In [64]:
import os
from PIL import Image, ImageSequence
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# ============ CONFIG ============ #
folder_path = 'test2'
model_path = 'best_model_resnet_version_2.pth'  # or EfficientNet path
model_type = 'resnet'  # or 'efficientnet'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size = 224
batch_size = 1

# Class names (adjust if needed)
class_names = [
    'Automatic Rifle', 'Bazooka', 'Grenade Launcher', 'Handgun', 'Knife',
    'Shotgun', 'SMG', 'Sniper', 'Sword'
]

# ============ TRANSFORM ============ #
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ============ DATASET CLASS ============ #
class UnlabeledImageDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.image_paths = sorted([
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if f.lower().endswith(('.jpg', '.jpeg', '.png', '.gif'))
        ])
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def load_gif_first_frame(self, path):
        with Image.open(path) as im:
            frame = next(ImageSequence.Iterator(im)).convert('RGB')
        return frame

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        if path.lower().endswith('.gif'):
            image = self.load_gif_first_frame(path)
        else:
            image = Image.open(path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, os.path.basename(path)

# ============ MODEL LOADING ============ #
from torchvision import models

def build_model(model_type="resnet", num_classes=9):
    if model_type == "resnet":
        model = models.resnet50(pretrained=False)
        model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    elif model_type == "efficientnet":
        from torchvision.models import efficientnet_b0
        model = efficientnet_b0(pretrained=False)
        model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)
    else:
        raise ValueError("Unsupported model type.")
    return model

model = build_model(model_type=model_type, num_classes=len(class_names))
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# ============ PREDICTION ============ #
dataset = UnlabeledImageDataset(folder_path, transform=transform)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

print("🔎 Predictions:")
with torch.no_grad():
    for inputs, filenames in loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1)
        predicted_class = class_names[preds.item()]
        confidence = probs[0][preds.item()].item() * 100
        print(f"{filenames[0]} → {predicted_class} ({confidence:.2f}%)")


🔎 Predictions:
3d Model Bazooka Rocket Launcher.png → Sword (21.67%)
Handgun _ Self-defense, Concealed Carry, Home Protection _ Britannica.jpg → Handgun (21.93%)
Image.jpg → Handgun (18.83%)


# HyperParameter Tuning

In [66]:
from itertools import product

learning_rates = [1e-3, 1e-4]
optimizers = ['adam', 'sgd']
batch_sizes = [32, 64]

best_acc = 0.0
best_params = {}

for lr, opt_name, bs in product(learning_rates, optimizers, batch_sizes):
    print(f"Training with lr={lr}, optimizer={opt_name}, batch_size={bs}")
    
    # Update dataloaders
    dataloaders = {
        x: DataLoader(image_datasets[x], batch_size=bs, shuffle=True)
        for x in ['train', 'val']
    }

    # Build model
    model = build_model(model_type=model_type, num_classes=num_classes).to(device)

    # Freeze backbone (if needed)
    for param in model.parameters():
        param.requires_grad = False
    if model_type == "resnet":
        for param in model.fc.parameters():
            param.requires_grad = True
    elif model_type == "efficientnet":
        for param in model.classifier.parameters():
            param.requires_grad = True

    # Create optimizer
    if opt_name == 'adam':
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    elif opt_name == 'sgd':
        optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=0.9)

    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    model = train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=num_epochs)

    # Evaluate on validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = correct / total
    print(f"Validation Accuracy: {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_params = {'lr': lr, 'optimizer': opt_name, 'batch_size': bs}
        torch.save(model.state_dict(), "best_model_tuned.pth")

print("\nBest Hyperparameters:")
print(best_params)
print(f"Best Validation Accuracy: {best_acc:.4f}")


Training with lr=0.001, optimizer=adam, batch_size=32

Epoch 1/20
------------------------------


100%|██████████| 31/31 [00:09<00:00,  3.20it/s]


Train Loss: 2.4958 Acc: 0.1186


100%|██████████| 4/4 [00:01<00:00,  2.41it/s]


Val Loss: 2.4662 Acc: 0.1359

Epoch 2/20
------------------------------


...

Val Loss: 2.1847 Acc: 0.1068

Epoch 20/20
------------------------------


100%|██████████| 16/16 [00:08<00:00,  1.84it/s]


Train Loss: 2.1948 Acc: 0.1155


100%|██████████| 2/2 [00:01<00:00,  1.29it/s]


Val Loss: 2.2011 Acc: 0.1068

Best Validation Acc: 0.1359
Validation Accuracy: 0.1068

Best Hyperparameters:
{'lr': 0.001, 'optimizer': 'adam', 'batch_size': 32}
Best Validation Accuracy: 0.2136


In [67]:
print("\nBest Hyperparameters:")
print(best_params)
print(f"Best Validation Accuracy: {best_acc:.4f}")


Best Hyperparameters:
{'lr': 0.001, 'optimizer': 'adam', 'batch_size': 32}
Best Validation Accuracy: 0.2136
